In [2]:
using Multitaper, JLD



# Simulate some data:
standd = [(0.01, 0.025)]
travel = [(0.1, 0.04)]
spow   = [2.0]
tpow   = [1.0]
Ns     = size(sleps[1])
m,n    = Ns
Ts     = Float64.(Ns)
Mx = 4*m; Mt = 4*n;


UndefVarError: UndefVarError: sleps not defined

In [ ]:

using MultiSpec, JLD, Rsvg
# DIR = "/home/cg/Work/data/rectsleps_laminar_K7.0.jld"
DIR = homedir()*"/Documents/Repos/spt/CLH_Code/"
FIGDIR = homedir()*"/Documents/Repos/spt/Figures/"

fn ="rectsleps_laminar_K7.0.jld"
sleps = load(DIR*fn)["sleps"]

include("heatmap_hayashi_pltlyjs.jl")
# Simulate some data:
standd = [(0.01, 0.025)]
travel = [(0.1, 0.04)]
spow   = [2.0]
tpow   = [1.0]
Ns     = size(sleps[1])
m,n=Ns
Ts     = Float64.(Ns)
simd   = MultiSpec.StandTravel(Ns..., Ts..., 1.0, standd, spow, travel, tpow)
Mx = 4*m; Mt = 4*n;

sleps = map(x->x./vecnorm(x),sleps)
# Compute its Hayashi decomp and pgram:
hay    = HayashiDecomposition(simd, 7.0, 2,Mx,Mt, Slep=sleps)
pgr    = HayashiDecomposition(simd, 7.0, 1,Mx,Mt, Slep=[ones(Ns...)./vecnorm(ones(Ns...))])

using PlotlyJS
# Plot the data
ix=32; iy=64
s = LinSpace.(0,1,m)
t = LinSpace.(0,1,n)

x   = MultiSpec.StandTravel(Ns..., Ts..., 0.0, standd,[0.0],travel,tpow)
y   = MultiSpec.StandTravel(Ns..., Ts..., 0.0, standd,spow,travel,[0.0])

#
st_trace = surface(x=s[1:16],y=t[1:32],z=x[1:16,1:32],colorscale="Viridis",opacity=0.95,title="Standing")
tr_trace = surface(x=s[1:ix],y=t[1:iy],z=y[1:ix,1:iy],colorscale="Viridis",opacity=0.95,title="Traveling")
nz_trace = surface(x=s[1:ix],y=t[1:iy],z=simd[1:ix,1:iy],colorscale="Viridis",opacity=0.95,title="Sum")

# Adjust camera angle
cam = attr(;up=attr(;x=0, y=0, z=1),center=attr(;x=0, y=0, z=0),
      	    eye=attr(;x=-1.5, y=-1.5, z=1.5))
    
l_o = Layout(;height=400,width=400,font_size=12,margin=attr(;l=50,r=50,b=75,t=75,pad=2),
	     aspectratio=attr(x=1, y=1, z=0.6),aspectmode="manual",scene=attr(;camera=cam),
	     xaxis_title="Space",yaxis_title="Time")

p_st = plot(st_trace,l_o) 
PlotlyJS.savefig(p_st,FIGDIR*"StandingWave.eps")
p_tr = plot(tr_trace,l_o)
PlotlyJS.savefig(p_tr,FIGDIR*"TravelingWave.eps")
#

w = linspace(0,0.5,Mx/2) ;
f = linspace(-0.5,0.5,Mt);

Wavecut = 0.07
Freqcut = 0.15

include("heatmap_hayashi_pltlyjs.jl")
seps = [maximum(pgr.traveling),maximum(pgr.standing)]
p_pgram  = hplot(pgr,f,w,Wavecut,Freqcut,stp=false,sep=seps) # standing phase off

seps = [maximum(hay.traveling),maximum(hay.standing)]
p_dpss  = hplot(hay,f,w,Wavecut,Freqcut,stp=false,sep=seps) # standing phase off

PlotlyJS.savefig(plot(p_pgram),FIGDIR*"Simplehaypgram.eps")
PlotlyJS.savefig(plot(p_dpss),FIGDIR*"Simplehaygram.eps")
